In [25]:
using KLIEPInference
using ProximalBase, CoordinateDescent
using LinearAlgebra, SparseArrays, Statistics, Random
using Distributions, JLD

In [18]:
function spKLIEP(Ψx, Ψy, λ; loadings=true)
    x = SparseIterate(size(Ψx, 1))
    f = KLIEPInference.CDKLIEPLoss(Ψx, Ψy)
    
    γ = _compute_loadings(x, Ψx, Ψy)
    g = ProxL1(λ, γ)
    coordinateDescent!(x, f, g)
    
    γ = _compute_loadings(x, Ψx, Ψy)
    g = ProxL1(λ, γ)
    coordinateDescent!(x, f, g, CDOptions(; warmStart=true))
    
    return x
end

function _compute_loadings(θ, ω, Ψx, Ψy)
    p, nx = size(Ψx)
    ny = size(Ψy, 2)

    wy = zeros(ny)
    mul!(wy, transpose(Ψy), θ)
    wy .= exp.(wy)
    wy ./= mean(wy)

    Ψyw = zeros(p, ny)
    for j = 1:ny
        for k = 1:p
            Ψyw[k, j] = wy[j] * Ψy[k, j]
        end
    end

    S  = cov( transpose(Ψx) , corrected=false) / nx
    S += cov( transpose(Ψyw), corrected=false) / ny
    
    return sqrt.(diag(S))
end

_compute_loadings (generic function with 1 method)

In [13]:
Random.seed!(1217)

MersenneTwister(UInt32[0x000004c1], Random.DSFMT.DSFMT_state(Int32[1375264445, 1073557489, 1793484471, 1073579540, 1202775888, 1073444522, 1951507022, 1073169694, 482126860, 1073209904  …  -1273351600, 1073496712, 1948612055, 1072718086, -2015603773, 817058167, -1504403733, -1290297799, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000

In [6]:
m = 25  #number of nodes
p = div(m * (m-1), 2)  #number of parameters
nx = 150
ny = 300

300

In [7]:
idx = KLIEPInference.trimap(5,6)  #edge of interest

15

In [10]:
file = jldopen("/Users/byolkim/Documents/KLIEPInference.jl/exper/exper1/graphs/chain1_25.jld", "r")
θx = pack(read(file, "Θx"))
θy = pack(read(file, "Θy"))
close(file)

spl = IsingSampler(θx; thin=2000)
X = rand(spl, nx)
spl = IsingSampler(θy; thin=2000)
Y = rand(spl, ny)

Ψx = Ψising(X)
Ψy = Ψising(Y)

300×300 Array{Float64,2}:
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  …  -1.0  -1.0   1.0  -1.0  -1.0  -1.0
  1.0   1.0   1.0   1.0   1.0   1.0      1.0   1.0   1.0   1.0   1.0   1.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0     -1.0  -1.0   1.0  -1.0  -1.0  -1.0
  1.0   1.0  -1.0   1.0   1.0  -1.0      1.0  -1.0   1.0  -1.0   1.0   1.0
 -1.0  -1.0   1.0  -1.0  -1.0   1.0     -1.0   1.0   1.0   1.0  -1.0  -1.0
  1.0   1.0  -1.0   1.0   1.0  -1.0  …   1.0  -1.0   1.0  -1.0   1.0   1.0
 -1.0   1.0  -1.0  -1.0   1.0  -1.0     -1.0  -1.0   1.0  -1.0   1.0   1.0
  1.0  -1.0   1.0   1.0  -1.0   1.0      1.0   1.0   1.0   1.0  -1.0  -1.0
 -1.0   1.0  -1.0  -1.0   1.0  -1.0     -1.0  -1.0   1.0  -1.0   1.0   1.0
 -1.0   1.0   1.0  -1.0   1.0   1.0     -1.0   1.0   1.0   1.0   1.0   1.0
  1.0   1.0  -1.0   1.0   1.0  -1.0  …  -1.0   1.0   1.0  -1.0  -1.0   1.0
 -1.0  -1.0   1.0  -1.0  -1.0   1.0      1.0  -1.0   1.0   1.0   1.0  -1.0
  1.0   1.0  -1.0   1.0   1.0  -1.0     -1.0   1.0   1.0  -1.0  -1.0   1.0

In [33]:
@show λ1 = 1.01 * quantile(Normal(), 1. - 0.05 / p)
θ = spKLIEP(Ψx, Ψy, λ1; loadings=true)

@show λ2 = sqrt(2. * log(p) / ny)
H = KLIEP_Hessian(θ, Ψy)
ω = Hinv_row(H, idx, λ2)

@show θ1, θ2 = debias_KLIEP(idx, θ, ω, Ψx, Ψy)

λ1 = 1.01 * quantile(Normal(), 1.0 - 0.05 / p) = 3.6237938190108476
λ2 = sqrt((2.0 * log(p)) / ny) = 0.19500055512153808
(θ1, θ2) = debias_KLIEP(idx, θ, ω, Ψx, Ψy) = (-0.036020991860404555, -0.05910655455443981)


(-0.036020991860404555, -0.05910655455443981)

In [49]:
δ = zeros(p)
δ[idx] = -1.0
δ = δ .+ 1.0
δ[idx]

0.0